In [18]:
import numpy as np
import pandas as pd
import xgboost as xgb

In [19]:
x = np.random.normal(0, 1, 100)
y = np.random.normal(0, 1, 100)
np.corrcoef(x, y)[0, 1]

-0.1366821462032053

In [20]:
def corr(x, y):
    xmean = np.mean(x)
    ymean = np.mean(y)
    xstd = np.std(x)
    ystd = np.std(y)
    return np.mean((x - xmean) * (y - ymean)) / (xstd * ystd)

In [31]:
corr(x, y)

-0.13668214620320546

In [32]:
def grad_corr(x, y):
    n = len(x)
    cov = np.mean((x - np.mean(x)) * (y - np.mean(y)))
    stdx = np.std(x)
    stdy = np.std(y)
    part1 = (y-np.mean(y))/(n*stdx*stdy)
    part2 = stdy**2/(stdx*stdy)**3*cov*(x-np.mean(x))
    return part1-part2

In [33]:
grad_corr(x, y)

array([ 0.13463657, -0.15240389, -0.08712631,  0.15442464, -0.20983227,
        0.14456403, -0.1344945 , -0.02311691, -0.08069219,  0.0931367 ,
        0.21768878, -0.08462913, -0.33625114,  0.29207215, -0.14708539,
       -0.01756962,  0.14805512,  0.14821924,  0.19838997,  0.00421451,
        0.00330911, -0.01423013, -0.09072364,  0.14807131,  0.27221636,
        0.12907284, -0.00463756, -0.01490361,  0.07358866, -0.22600293,
       -0.10741577, -0.11000171, -0.04505117,  0.06474243,  0.045628  ,
       -0.02662797, -0.01981014,  0.02961087,  0.14662018, -0.27168922,
        0.11048939, -0.1226482 ,  0.32827295,  0.11444837, -0.16140303,
       -0.02353132,  0.07573237,  0.05752561,  0.11709348, -0.16154203,
       -0.01083168, -0.07791917,  0.09461653, -0.34618887,  0.15891815,
       -0.29990729, -0.07740516,  0.29638123, -0.16175911,  0.24625698,
        0.00953391,  0.20415147,  0.1257433 , -0.22687105, -0.03453165,
       -0.19913869, -0.02624465, -0.0470936 ,  0.10718249, -0.08

In [38]:
def grad_corr2(x, y):
    B = np.sum((x - np.mean(x)) * (y - np.mean(y)))
    C = np.sum((y - np.mean(y))**2)
    D = np.sum((x - np.mean(x))**2)
    return ((y - np.mean(y)) - B/D*(x - np.mean(x)))/np.sqrt(C*D)

In [39]:
grad_corr2(x, y)

array([-0.00223605,  0.01717174,  0.01548725,  0.00025399, -0.0064104 ,
       -0.01619607,  0.00796994, -0.00019612, -0.00796816,  0.01998275,
       -0.0045301 ,  0.00539953,  0.00086094, -0.00503676,  0.00346135,
        0.01609242, -0.01895355, -0.01795338,  0.00885745, -0.0043214 ,
        0.00105036,  0.00450743, -0.00057535, -0.01556991,  0.01276231,
       -0.00113471,  0.00479116,  0.00407751, -0.00273056,  0.00101463,
        0.00666294, -0.00630334,  0.02539273, -0.01460568, -0.02219062,
        0.00494189, -0.0081689 , -0.00358018, -0.0145535 ,  0.00274156,
        0.03905719,  0.00488755,  0.00028309, -0.01037268,  0.00176734,
       -0.00597374, -0.00610292, -0.01764759, -0.00538243, -0.00615395,
       -0.00110178,  0.01546743, -0.00613899, -0.0088597 ,  0.00661046,
        0.00419256, -0.01497067,  0.00258986, -0.00569489, -0.00944751,
        0.00266503, -0.00672115, -0.00129964, -0.00987621, -0.01331077,
        0.00231745,  0.01019426, -0.00477431,  0.01504962,  0.00

In [24]:
def hessian_corr(x, y):
    n = len(x)
    varx = np.mean((x-np.mean(x))**2)
    vary = np.mean((y-np.mean(y))**2)
    part1 = -(y-np.mean(y))*vary/(np.sqrt(varx*vary)**3)*(x-np.mean(x))
    cov = np.mean((x-np.mean(x))*(y-np.mean(y)))
    h = vary/(np.sqrt(varx*vary)**3)*(x - np.mean(x))
    part2 = -(h/n*(y-np.mean(y))+cov*((1-1/n)*vary/(np.sqrt(varx*vary)**3)-3/n*vary**2/(np.sqrt(varx*vary)**5)*(x-np.mean(x))))
    return part1 + part2

In [25]:
hessian_corr(x, y)

array([ 0.5293761 ,  2.5612626 ,  1.43410061,  0.3090265 , -0.48909259,
        2.29705844,  1.17051033,  0.16619165, -0.22522628, -0.88871844,
        1.28023532,  0.59200657,  1.25062466,  1.93628352,  0.73011841,
        0.57885583,  2.73154269,  2.59359258, -0.8244609 ,  0.19250753,
        0.16292473,  0.23111882,  0.19062368,  2.26234683, -1.80882286,
        0.39804036,  0.19947699,  0.22592555,  0.36138543,  0.73360247,
        0.83440922, -0.23824435,  1.53989376,  1.07431581,  1.32020749,
        0.28986942,  0.09521146,  0.26051685,  2.10544311,  1.30424858,
       -1.88184947,  0.75733959,  0.89440901,  1.24399424,  0.58674157,
        0.08942608,  0.58499289,  1.19477236,  0.76535981, -0.36123385,
        0.15781158,  1.31208261,  0.69924024, -1.19562616, -0.41747927,
        1.82481658, -0.5012211 ,  0.23673274, -0.3092515 ,  2.45141438,
        0.15117925,  1.55010366,  0.40497853, -1.07185624, -0.04191561,
        0.82662715,  0.45336089,  0.02899667, -0.80959873,  0.38

In [28]:
def corr_loss(predt: np.ndarray, dtrain: xgb.DMatrix) -> tuple[np.ndarray, np.ndarray]:
    def grad_corr(x, y):
        n = len(x)
        cov = np.mean((x - np.mean(x)) * (y - np.mean(y)))
        stdx = np.std(x)
        stdy = np.std(y)
        part1 = (y-np.mean(y))/(n*stdx*stdy)
        part2 = stdy**2/(stdx*stdy)**3*cov*(x-np.mean(x))
        return part1-part2
    
    def hessian_corr(x, y):
        n = len(x)
        varx = np.mean((x-np.mean(x))**2)
        vary = np.mean((y-np.mean(y))**2)
        part1 = -(y-np.mean(y))*vary/(np.sqrt(varx*vary)**3)*(x-np.mean(x))
        cov = np.mean((x-np.mean(x))*(y-np.mean(y)))
        h = vary/(np.sqrt(varx*vary)**3)*(x - np.mean(x))
        part2 = -(h/n*(y-np.mean(y))+cov*((1-1/n)*vary/(np.sqrt(varx*vary)**3)-3/n*vary**2/(np.sqrt(varx*vary)**5)*(x-np.mean(x))))
        return part1 + part2
        
    y = dtrain.get_label()
    grad = grad_corr(predt, y)
    hess = hessian_corr(predt, y)
    return grad, hess

In [50]:
eval_metric = "rmse"
objective = "reg:squarederror"
n_estimators = 200
max_depth = 2
learning_rate = 0.1
#early_stopping_rounds = 1000

min_child_weight = 4
subsample = 0.5
colsample_bytree = 0.5
gamma = 0
reg_alpha = 0
reg_lambda = 0
seed = 10
verbosity = 3

In [51]:
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

data_url = "http://lib.stat.cmu.edu/datasets/boston"
raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
X = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
y = raw_df.values[1::2, 2]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

xmean = np.mean(X_train, axis=0)
xstd = np.std(X_train, axis=0)
ymean = np.mean(y_train, axis=0)
ystd = np.std(y_train, axis=0)

print("%-20s %-20s %-20s %-20s" % ("train_MSE", "valid_MSE", "train_IC", "valid_IC"))
xgb_model = XGBRegressor(seed=seed,
                     n_estimators=n_estimators,
                     max_depth=max_depth,
                     learning_rate=learning_rate,
                     min_child_weight=min_child_weight,
                     subsample=subsample,
                     colsample_bytree=colsample_bytree,
                     gamma=gamma,
                     reg_alpha=reg_alpha,
                     reg_lambda=reg_lambda,
                     verbosity=0,
                     #nthread=nthread,
                     eval_metric=eval_metric,
                     objective=objective,
                     )
xgb_model.fit((X_train-xmean)/xstd, (y_train-ymean)/ystd)
train_pred = xgb_model.predict((X_train-xmean)/xstd)*ystd+ymean
valid_pred = xgb_model.predict((X_test-xmean)/xstd)*ystd+ymean
print("%-20.15f %-20.15f %-20.15f %-20.15f" % (mean_squared_error(y_train, train_pred), mean_squared_error(y_test, valid_pred), cal_ic_raw(y_train, train_pred), cal_ic_raw(y_test, valid_pred)))

train_MSE            valid_MSE            train_IC             valid_IC            


NameError: name 'mean_squared_error' is not defined